# Welcome to AmpForm!

:::{title} Welcome
:::

```{autolink-concat}

```

[![10.5281/zenodo.5526648](https://zenodo.org/badge/doi/10.5281/zenodo.5526648.svg)](https://doi.org/10.5281/zenodo.5526648)
[![Supported Python versions](https://img.shields.io/pypi/pyversions/ampform)](https://pypi.org/project/ampform)
{{ '[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ComPWA/ampform/blob/{})'.format(branch) }}
{{ '[![Binder](https://static.mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ComPWA/ampform/{}?urlpath=lab)'.format(branch) }}

AmpForm automatically formulates amplitude models for arbitrary particle transitions in order to do a [partial-waves analysis](https://pwa.rtfd.io). All allowed transitions between some initial and final state are generated with the {mod}`qrules` package. You can then use {mod}`ampform` to formulate these state transitions into an amplitude model with some formalism, like the helicity formalism.

::::{grid} 1 2 2 2
:gutter: 2

:::{grid-item-card} Flexible partial-wave decomposition
:link: amplitude
:link-type: doc
Scan which resonances are allowed for a given particle decay, then formulate the amplitude model automatically with the spin formalism of your choice.
:::

:::{grid-item-card} Narrow the gap between code and theory
:link: https://compwa.github.io/symbolics
AmpForm amplitude models are formulated symbolically with SymPy, so that you can easily inspect the mathematical expressions and modify then algebraically.
:::

:::{grid-item-card} Adjust and modify the model to your needs
:link: dynamics/custom
:link-type: doc
Customize dynamics lineshapes in the amplitude builder, for instance if you needsome additional background terms, without having to tweak the entire framework.
:::

:::{grid-item-card} Convert to several computational back-ends
:link: https://tensorwaves.rtfd.io
The amplitude models are converted to computational back-ends like JAX, TensorFlow, and NumPy. The TensorWaves package facilitates this and also provides tools to generate toy Monte Carlo data samples.
:::

:::{grid-item-card} {fab}`python` Use your favorite Python packages
:columns: 12
:link: api/ampform
:link-type: doc
All this functionality is offered in an open and transparent style, so that you can navigate AmpForm's output and feed it to whatever other Python packages come in handy for you. For instance, check out inspect the model interactively to see how to inspect your model with an interactive widget.
:::

::::

The following pages provide more advanced examples of how to use AmpForm. You can run each of them as Jupyter notebooks with the {fa}`rocket` launch button in the top-right corner.

## Dynamics parametrizations

AmpForm offers a number of functions for parametrizing dynamics lineshapes. The functions are expressed with {mod}`sympy`, so that they can easily be visualized, simplified, or modified:

In [ ]:
import logging

from IPython.display import Math

from ampform.io import improve_latex_rendering

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)
improve_latex_rendering()

In [ ]:
import sympy as sp

from ampform.dynamics import relativistic_breit_wigner

m, m0, w0 = sp.symbols("m m0 Gamma0")
relativistic_breit_wigner(s=m**2, mass0=m0, gamma0=w0)

In [ ]:
from ampform.dynamics import relativistic_breit_wigner_with_ff

m1, m2, L = sp.symbols("m1 m2 L")
relativistic_breit_wigner_with_ff(
    s=m**2,
    mass0=m0,
    gamma0=w0,
    m_a=m1,
    m_b=m2,
    angular_momentum=L,
    meson_radius=1,
)

In [ ]:
from ampform.dynamics.kmatrix import NonRelativisticKMatrix

n_poles = sp.Symbol("n_R", integer=True, positive=True)
NonRelativisticKMatrix.formulate(
    n_poles=n_poles,
    n_channels=1,
)[0, 0]

In [ ]:
matrix = NonRelativisticKMatrix.formulate(n_poles=1, n_channels=2)
matrix[0, 0].doit().simplify()

More dynamics functions can be found in the {mod}`.dynamics` library, as well as on the {doc}`dynamics` page!

## Formulate amplitude models

Together with [QRules](https://qrules.rtfd.io), AmpForm can automatically formulate amplitude models for generic, multi-body decays. These models can then be used as templates for faster computational back-ends with [TensorWaves](https://tensorwaves.rtfd.io). Here's an example:

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state=("psi(4160)", [-1, +1]),
    final_state=["D-", "D0", "pi+"],
    allowed_intermediate_particles=["D*(2007)0"],
    formalism="helicity",
)

In [ ]:
import ampform
from ampform.dynamics.builder import create_relativistic_breit_wigner

builder = ampform.get_builder(reaction)
for particle in reaction.get_intermediate_particles():
    builder.dynamics.assign(particle.name, create_relativistic_breit_wigner)
model = builder.formulate()
model.intensity

In [ ]:
from ampform.io import aslatex

(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

In case of multiple decay topologies, AmpForm also takes care of {doc}`spin alignment <amplitude/spin-alignment>` with {cite}`Marangotto:2019ucc`!

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="Lambda(c)+",
    final_state=["p", "K-", "pi+"],
    allowed_intermediate_particles=[
        "Lambda(1405)",
        "Delta(1232)++",
    ],
)

In [ ]:
import graphviz

dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
builder = ampform.get_builder(reaction)
model = builder.formulate()
model.intensity

:::{tip}
Amplitude models for three-body decays that involve spin in the final state are best formulated with the [AmpForm-DPD](https://ampform-dpd.rtfd.io) extension.
:::

## Advanced examples

The following pages provide more advanced examples of how to use AmpForm.

```{toctree}
:maxdepth: 2
install
amplitude
dynamics
analyticity
kinematics
sympy
references
```

```{toctree}
:hidden:
:maxdepth: 2
API <api/ampform>
Continuous benchmarks <https://compwa.github.io/ampform-benchmark-results>
Changelog <https://github.com/ComPWA/ampform/releases>
Upcoming features <https://github.com/ComPWA/ampform/milestones?direction=asc&sort=title&state=open>
Help developing <https://compwa.github.io/develop>
```

```{toctree}
:caption: Related projects
:hidden:
QRules <https://qrules.readthedocs.io>
TensorWaves <https://tensorwaves.readthedocs.io>
PWA Pages <https://pwa.readthedocs.io>
ComPWA project <https://compwa.github.io>
```